In [3]:
#Importing Dependencies
import tkinter as tk
from tkinter import filedialog
import whisper
import deepl
import os
import torch
import moviepy.editor as mp

In [4]:
# Grabbing GPU for Whisper
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Your DeepL API key
DEEPL_API_KEY = "828320a4-c2ec-48d1-952e-6022a2bd8cd6:fx"

# Function to extract audio from a video file
def extract_audio_from_video(video_file_path):
    video = mp.VideoFileClip(video_file_path)
    audio_file_path = os.path.splitext(video_file_path)[0] + ".wav"
    video.audio.write_audiofile(audio_file_path)
    return audio_file_path

# Function to transcribe the audio
def transcribe_audio(file_path):
    model = whisper.load_model("medium")  # Load Whisper model (base version)
    
    # Transcribe the audio
    result = model.transcribe(file_path, language="en")
    
    return result['text']

# Function to translate the transcribed text
def translate_text(text, target_lang="EN-US"):
    translator = deepl.Translator(DEEPL_API_KEY)
    
    # Translate text using DeepL
    translated_text = translator.translate_text(text, target_lang=target_lang)
    
    return translated_text.text

# Function to save the translated text
def save_translation(translated_text, file_path, output_dir):
    file_name = os.path.basename(file_path).split('.')[0] + '_translated.txt'
    output_path = os.path.join(output_dir, file_name)
    
    with open(output_path, 'w') as f:
        f.write(translated_text)
    
    return output_path

# Function to handle file selection, transcription, and translation
def select_file():
    file_path = filedialog.askopenfilename(title="Select Audio/Video File", 
                                           filetypes=[("Media Files", "*.mp3 *.wav *.m4a *.ogg *.flac *.mp4 *.mkv *.avi")])
    if file_path:
        output_dir = filedialog.askdirectory(title="Select Output Directory")
        if output_dir:
            try:
                # Check if the selected file is a video
                if file_path.endswith((".mp4", ".mkv", ".avi")):
                    # Step 1: Extract audio from the video
                    audio_file_path = extract_audio_from_video(file_path)
                else:
                    # If it's already an audio file
                    audio_file_path = file_path

                # Step 2: Transcribe the audio
                transcribed_text = transcribe_audio(audio_file_path)

                # Step 3: Translate the transcription
                translated_text = translate_text(transcribed_text, target_lang="EN-US")

                # Step 4: Save the translation
                translated_file = save_translation(translated_text, file_path, output_dir)

                result_label.config(text=f"Translation saved: {translated_file}")

            except FileNotFoundError as e:
                print(f"FileNotFoundError: {e}")
                result_label.config(text="Error: File not found.")

            except Exception as e:
                print(f"Error: {e}")
                result_label.config(text="An unexpected error occurred.")
        else:
            result_label.config(text="Output directory not selected.")
    else:
        result_label.config(text="No file selected.")

# Create the tkinter window
root = tk.Tk()
root.title("Whisper + DeepL Transcriber & Translator")
root.geometry("400x200")

# Create the buttons and labels
select_button = tk.Button(root, text="Select Audio/Video File", command=select_file)
select_button.pack(pady=20)

result_label = tk.Label(root, text="")
result_label.pack(pady=20)

# Run the tkinter event loop
root.mainloop()

Using device: cuda
MoviePy - Writing audio in C:/Users/Cajun/Downloads/IS_Intro.wav


MoviePy - Done.
